# box plot 

# Load library

In [12]:
library(ggplot2)
library(Hmisc)
source("../utilis/utilis.R")
library(glue)
library(dplyr)

# Set values & table names

In [13]:
threshold = 0.05
cutoff_column = 'padj'
data_df_path =  '../../../output/PS_species_v2_hMinImp_TICnorm_groupFil0.3_HILICpos_1wayANOVA/one_wayANOVA_PS_species_v2_hMinImp_TICnorm_groupFil0.3_HILICpos_fullreport.csv'
meta_df_path =  '../../../input/sequence_file/clean_deduplicated_meta_df.csv'

# Prepare the data

## Read and transform the data table and metadata table

In [14]:
df = read.csv(data_df_path, row.names = 1, sep = ',')
meta_df = read.csv(meta_df_path, row.names = 1)

----

In [15]:
df = df %>% filter(!is.na(neutral_mass))

In [16]:
rownames(df)

[1] "F13199" "F15644" "F16887" "F17414" "F17672" "F17876" "F17923" "F18169"
 [9] "F18473" "F18893"

In [17]:
if (!(grepl("\\|",rownames(df)[1]))) {
    rownames(df) = paste0(rownames(df),"|",df$ion_relation,"|",df$short_name)
} 


In [18]:
grepl('Naive\\_[0-9]|R5pos\\_[0-9]|R5neg\\_[0-9]',colnames(df))

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[37] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE
[49] FALSE

In [19]:
data_df = df[grepl('Naive\\_[0-9]|R5pos\\_[0-9]|R5neg\\_[0-9]',colnames(df))]

In [20]:
head(data_df)

,Naive_1,Naive_2,Naive_3,R5neg_1,R5neg_2,R5neg_3,R5pos_1,R5pos_2,R5pos_3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"F13199|M0,M+H+|LPS 20:3",24.08338,24.45670,23.64044,24.37743,23.83728,24.03485,22.91200,24.82122,24.43848
"F15644|M0,M+H+|LPS O-17:0;O",23.00810,24.25440,24.76713,23.68646,23.98221,23.92793,25.25177,25.09492,24.15703
F16887||PS 37:5,21.02884,21.17845,21.22538,19.57986,20.57986,19.57986,21.25309,21.85145,21.46881
"F17414|M0,M+H+|PS 40:6",22.89911,22.48117,22.37244,24.13285,24.24680,24.09082,21.21226,22.78435,22.03645
"F17672|M0,M+H+|PS 42:9",17.15498,16.15498,16.15498,19.88481,20.25283,20.31554,16.15498,16.15498,16.15498
F17876||PS 42:9,21.05213,20.96355,20.42411,18.04788,20.15459,17.04788,17.04788,21.59848,21.23466


----

In [21]:
head(meta_df)

,class,biological_samples,cell_number
,<chr>,<chr>,<int>
Naive_1,Naive,Yes,300
Naive_2,Naive,Yes,300
Naive_3,Naive,Yes,300
R5neg_1,R5neg,Yes,655
R5neg_2,R5neg,Yes,687
R5neg_3,R5neg,Yes,865


----

## Transform and merge data

In [22]:
source('../utilis/utilis.R')
transform_merge_data4violin <- function(df = df,
                                 meta_df = meta_df,
                                 class_column = 'class',
                                 cutoff_column = 'padj', 
                                 threshold = threshold) {
    
    df_filt <- df[df[,cutoff_column] < threshold,
              colnames(df)[grepl('Naive\\_[0-9]|R5pos\\_[0-9]|R5neg\\_[0-9]',colnames(df))]] # 
    df_filt.T <- t(df_filt)
    # print(head(df_filt.T))
    merged_df = merge_by_rowName(meta_df,df_filt.T, all.y = TRUE)
    merged_df[,class_column] = factor(
        merged_df[,class_column],
        level = c('Naive',
                  'R5pos',
                  'R5neg')
    )
    return(merged_df)
}

In [23]:
merged_df <- transform_merge_data4violin(df = df,
                                         meta_df = meta_df,
                                         class_column = 'class',
                                         cutoff_column = 'padj', 
                                         threshold = threshold)

In [24]:
colnames(merged_df)

[1] "class"                  "biological_samples"     "cell_number"           
[4] "F16887||PS 37:5"        "F17414|M0,M+H+|PS 40:6" "F17672|M0,M+H+|PS 42:9"
[7] "F18473||PS 36:1"        "F18893|M0,Na/H|PS 36:1"

# Output directory

In [25]:
output_dir = "../../../output/PS_species_v2_hMinImp_TICnorm_groupFil0.3_HILICpos_1wayANOVA//boxplot/"
dir.create(output_dir)

# Wrapper Function: plot a single box plot

In [26]:
box_plot <- function(long_df = merged_df,
                        x = 'class',
                        y = var,
                        xlab = xlab,
                        ylab = ylab,
                        fill_color = 'class',
                        fig_width = 8,
                        fig_height = 8,
                        text_size = 20,
                        legend_size = 20,
                        axis_title_size = 20,
                        output_dir = "",
                        pdf_prefix = 'test',
                        show_plot = TRUE) {
    
    options(repr.plot.width = fig_width, repr.plot.height = fig_height)

    p <- ggplot(long_df, aes_string(x = x, y = y, fill = fill_color)) + 
        geom_boxplot(outlier.shape = NA) +
        xlab(xlab) + ylab(ylab) + 
        geom_jitter(alpha = 1) +
        scale_fill_manual(values = c("#888888","#0000ff","#ff0000")) +  #

        # set transparency
        # https://ggplot2.tidyverse.org/reference/theme.html
        theme(
          panel.grid.major = element_line(colour = "grey50",linetype = "dashed", size = 0.2),
          panel.border = element_rect(fill = NA),
          #panel.grid.minor = element_blank(),
          panel.background = element_rect(fill = "transparent",colour = NA),
          plot.background = element_rect(fill = "transparent",colour = NA),
          axis.text = element_text(size = text_size, colour="black"),
          axis.text.x = element_text(angle = 45, vjust = 0.5, hjust=0.5, colour="black"),
          legend.text= element_text(size = legend_size),
          legend.title = element_text(size = legend_size),
          axis.title = element_text(size = axis_title_size)
        )
    if (show_plot == TRUE) {
        print(p) # display the plot
    } else {return(p)}

    if (length(output_dir)!=0) {
            ggsave(file.path(output_dir, paste0("ggplot_boxplot_",pdf_prefix,".pdf")), width = fig_width, height = fig_height)
        }
}

# Plot a single box plot

In [27]:
# var <- sym('F10352')

# metab_name <- lookUpAnnot(annot_df,var,'Species.Shorthand')
# box_plot(long_df = merged_df,
#             x = 'class',
#             y = var,
#             xlab = 'class',
#             ylab = glue('{var}_{metab_name}\nlog2peakArea'),
#             fill_color = 'class',
#             fig_width = 8,
#             fig_height = 6,
#             text_size = 20,
#             legend_size = 20,
#             axis_title_size = 20,
#             output_dir = output_dir,
#             pdf_prefix = 'test')

# Wrapper for plotting out multiple

In [28]:
head(merged_df)

,class,biological_samples,cell_number,F16887||PS 37:5,"F17414|M0,M+H+|PS 40:6","F17672|M0,M+H+|PS 42:9",F18473||PS 36:1,"F18893|M0,Na/H|PS 36:1"
,<fct>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Naive_1,Naive,Yes,300,21.02884,22.89911,17.15498,18.27402,21.64989
Naive_2,Naive,Yes,300,21.17845,22.48117,16.15498,17.15197,21.06372
Naive_3,Naive,Yes,300,21.22538,22.37244,16.15498,17.75366,21.12758
R5neg_1,R5neg,Yes,655,19.57986,24.13285,19.88481,19.86190,21.76141
R5neg_2,R5neg,Yes,687,20.57986,24.24680,20.25283,19.25296,22.08037
R5neg_3,R5neg,Yes,865,19.57986,24.09082,20.31554,19.61891,21.79392


In [29]:
listOfFeatures = colnames(merged_df)[grepl('F[0-9]',colnames(merged_df))] # 

In [30]:
# install.packages("ggpubr")
library(ggpubr)

In [33]:
fig_l = list()
for (i in 1:length(listOfFeatures)) {

    pdf_prefix = paste('test',cutoff_column,threshold, sep = "_") # 

    featID <- listOfFeatures[[i]]

    fig_l[[i]] <- box_plot(long_df = merged_df,
                            x = 'class', #
                            y = sym(featID),
                            xlab = 'class', # 
                            ylab = glue('{featID}\nlog2peakArea'),
                            fill_color = 'class',
                            fig_width = 4,
                            fig_height = 4,
                            text_size = 8, # important in report multiple ones.
                            legend_size = 8,
                            axis_title_size = 6,
                            output_dir = NULL, # no need to define, this is for single plot
                            pdf_prefix = NULL, # no need to define, this is for single plot
                            show_plot = FALSE)
                    }
multi.page <- ggarrange(plotlist = fig_l,
                        nrow = 3, ncol = 2)
ggexport(multi.page, filename = file.path(output_dir, 
                                          paste0("ggplot_multi_ggplot_",
                                                 pdf_prefix,
                                                 ".pdf")))


# library(ggpubr)
# 
# my_comparisons = list( c("0.5", "1"), c("1", "2"), c("0.5", "2") )
# 
# ggboxplot(ToothGrowth, x = "dose", y = "len",
#           color = "dose", palette = "jco")+ 
#   stat_compare_means(comparisons = my_comparisons, label.y = c(29, 35, 40))+
#   stat_compare_means(label.y = 45)

Warning message:
“`aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation ideoms with `aes()`”
Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”
file saved to ../../../output/PS_species_v2_hMinImp_TICnorm_groupFil0.3_HILICpos_1wayANOVA//boxplot//ggplot_multi_ggplot_test_padj_0.05.pdf



-------

-------

-------